In [ ]:
from kaggle.api.kaggle_api_extended import KaggleApi
import tempfile
import os
import csv

api = KaggleApi()
api.authenticate()

num_datasets = 50

search_results = api.dataset_list(search="")

df_dict = {}

for i, dataset in enumerate(search_results[:num_datasets]):
    try:
        dataset_name = dataset.ref
        dataset_files = api.dataset_list_files(dataset_name).files

        csv_file = next((file.name for file in dataset_files if file.name.endswith('.csv')), None)

        if csv_file:
            with tempfile.TemporaryDirectory() as temp_dir:
                api.dataset_download_file(dataset_name, csv_file, path=temp_dir, quiet=True)

                file_path = os.path.join(temp_dir, csv_file)

                with open(file_path, newline='') as csvfile:
                    df_dict[f"dataset_{i+1}"] = list(csv.DictReader(csvfile))

                print(f"Loaded dataset {i+1}: {dataset_name}")
        else:
            print(f"No CSV file found in dataset {dataset_name}")

    except Exception as e:
        print(f"Error processing dataset {dataset_name}: {e}")

print("Datasets loaded into df_dict:")
print(df_dict.keys())


In [ ]:
d = {}
for k, v in df_dict.items():
    d[k] = df_dict[k]

In [ ]:
converted_data = {}
for dataset_name, dataset_list in d.items():
    if dataset_list:
        columns = dataset_list[0].keys()
        converted_data[dataset_name] = {
            col: [item[col] for item in dataset_list]
            for col in columns
        }

In [ ]:
import pickle

In [ ]:
with open('data.pkl', 'wb') as file:  # Note: 'wb' for binary write
    pickle.dump(converted_data, file)